## Popular Tablas

Con las tablas **persona** y **favorite_food** ya creadas, ahora podemos comenzar a explorar las formas de trabajar con los datos:

### Insertar datos en tablas

Dado que todavía no hay ningún dato en las tablas **persona** y **favorite_food**, vamos a explorar la declaración de inserción: _**INSERT**_. Hay tres componentes principales en una declaración _**INSERT**_:

- Nombre de la tabla en la que agregamos los datos.

- Los nombres de las columnas de la tabla que se cumplimentarán.

- Los valores con los cuales se rellenarán las columnas.

Su sintaxis es la siguiente:

```mysql
INSERT INTO nombre_tabla [ '(' columnas ')' ]
 { VALUES '(' {valores} ')', };
```


- No es necesario que proporcionemos datos para cada columna de la tabla (a menos que todas las columnas de la tabla se hayan definido como **NOT NULL**).
<br>

- En algunos casos, aquellas columnas que no están incluidas en la declaración de _**INSERT**_ inicial recibirán un valor más adelante a través de una declaración de actualización: _**UPDATE**_.
<br>

- En otros casos, es posible que una columna nunca reciba un valor para una fila de datos en particular (como un pedido de un cliente que se cancela antes de enviarse).

___

### Generación de datos de clave numérica

Antes de insertar datos en la tabla persona, sería útil analizar cómo se generan los valores para las primary keys numéricas. Para insertar valores en las primarys keys numéricas tenemos dos opciones:

- Observar cual es el valor más alto del campo que es la primary key y sumarle uno manualmente.

- Dejar que el servidor de la base de datos nos proporcione el valor por nosotros.

Aunque la primera opción nos pueda parecer válida, es problemática en un entorno multiusuario ya que dos usuarios pueden mirar la tabla al mismo tiempo y generar el mismo valor para la primary key (Cuando esta debe ser valor único).

MySQL, nos permite activar la función **AUTO_INCREMENT** para su columna de primary key.

Como ya tenemos las tablas generadas y ese constraint ya está en la columna de primary key no es necesario modificar la tabla.

En el caso de que este constraint no exista podemos usar la declaración **ALTER TABLE** que la usamos para modificar la definición de una tabla existente, aquí cómo modificaríamos nuestra tabla persona para asignarle **AUTO_INCREMENT** a nuestra primary key **persona_id** en caso de no haberlo hecho:

```mysql
 ALTER TABLE persona MODIFY persona_id SMALLINT UNSIGNED AUTO_INCREMENT;
```

Pero para añadir esta declaración a nuestra base de datos, primero debemos deshabilitar (temporalmente) la restricción de la clave externa en la tabla **favorite_food** (o no nos admitirá el cambio) y luego volver a habilitarlas restricciones al acabar (importante comprobar que las restricciones quedan habilitadas), entonces la sentencia de _**ALTER**_ anterior quedaría así:

```mysql
 set foreign_key_checks=0;

 ALTER TABLE persona MODIFY persona_id SMALLINT UNSIGNED AUTO_INCREMENT;

 set foreign_key_checks=1;
```
___

Ahora, cuando insertemos datos en la tabla persona, simplemente proporcionamos un valor nulo para la columna **person_id** y MySQL completará la columna con el siguiente número disponible (de forma predeterminada, **MySQL** comienza en 1 para las columnas de incremento automático).

Volvemos con la declaración _**INSERT**_, vamos a ingresar nuestro primer registro. La siguiente instrucción crea una fila en la tabla persona para _'William Turner'_:

```mysql
 INSERT INTO persona (persona_id, fname, lname, eye_color, birth_date)
    VALUES (null, 'William', 'Turner', 'BR', '1972-05-27');
```

Para comprobar que la tabla **persona** tiene ahora una fila podemos ejecutar la siguiente query:

```mysql
 SELECT
     persona_id, fname, lname, birth_date
 FROM
     persona;

```

Como podemos comprobar, el servidor **MySQL** generó un valor de 1 para la primary key.
___

Antes de continuar, vale la pena mencionar un par de cosas sobre la anterior declaración de INSERT:

- No proporcionamos valores para ninguna de las columnas relativas a la dirección (**street**, **city**, **state**, **country**, **postal_code**). Esto está bien, ya que se permiten nulos para esas columnas.
<br>

- El valor proporcionado para la columna **birth_date** era una string. Siempre que coincida con el formato requerido que se muestra en la tabla de tipos de datos temporales, **MySQL** convertirá el string en un date.
<br>

- Los nombres de las columnas y los valores proporcionados deben corresponder en número y tipo. Si nombra siete columnas y proporciona solo seis valores o si proporciona valores que no se pueden convertir al tipo de datos apropiado para la columna correspondiente, recibirá un error.

Ahora vamos a insertar la información sobre los tres alimentos favoritos de _'William Turner'_ por medio de tres declaraciones de _**INSERT**_ en la tabla **favorite_food**:

```mysql
 INSERT INTO favorite_food (persona_id, food)
 VALUES (1, 'pizza');

 INSERT INTO favorite_food (persona_id, food)
 VALUES (1, 'cookies');

 INSERT INTO favorite_food (persona_id, food)
 VALUES (1, 'nachos');
```

También podemos ejecutarlo de esta forma:


```mysql
 INSERT INTO favorite_food (persona_id, food)
 VALUES (1, 'pizza'),
        (1, 'cookies'),
        (1, 'nachos');
```


Para comprobar que la tabla **favorite_food** tiene ahora una fila podemos ejecutar la siguiente query:

```mysql
 SELECT
     persona_id, food
 FROM
     favorite_food;

```
___

## Modificar Datos en Tablas

La modificación de registros ya insertados en la tabla se realiza con la sentencia _**UPDATE**_, que tiene el siguiente formato:

```mysql
 UPDATE nombre_tabla
 SET columna = valor [ {, columna = valor}]
  [ WHERE condiciones ]
```
___

Cuando los datos de _'William Turner'_ se agregaron inicialmente a la tabla, los datos de las diversas columnas de dirección no fueron incluidas en la declaración _**INSERT**_. La siguiente declaración muestra cómo estas columnas se pueden completar en un momento posterior a través de una declaración _**UPDATE**_:

```mysql
 UPDATE persona
 SET street = '1225 Tremont St.',
     city = 'Boston',
     state = 'MA',
     country = 'USA',
     postal_code = '02138'
 WHERE persona_id = 1;
```

El servidor respondió con un mensaje de dos líneas: el elemento _"Rows matched: 1"_ nos dice que la condición en la cláusula WHERE coincidió con una sola fila en la tabla, y el elemento _"Changed: 1"_ nos dice que una sola fila en la tabla ha sido modificada.

Dado que la cláusula _**WHERE**_ especifica la primary key correspondiente a la fila de _'William'_, esto es exactamente lo que esperaríamos que sucediera.

Dependiendo de las condiciones en su cláusula _**WHERE**_, también es posible modificar más de una fila usando una sola declaración. Consideremos, por ejemplo, qué sucedería si nuestra cláusula _**WHERE**_ tuviera el siguiente aspecto:

```mysql
 WHERE persona_id < 10;
```

En el caso de que en la columna **persona_id** existieran más valores se modificarían todas las filas que cumplan con la condición.

**Si omitimos la cláusula _WHERE_ por completo, la instrucción _UPDATE_ modificará todas las filas de la tabla.**

## Eliminación de Datos en Tablas

El borrado de filas de una tabla se efectúa con la sentencia _**DELETE**_ con la siguiente sintaxis:

```mysql
 DELETE FROM nombre_tabla
  [ WHERE condiciones ]
```
 
 
Si quisieramos eliminar el único registro de la tabla podemos hacer:

```mysql
 DELETE FROM persona
 WHERE person_id = 1;
```

Nuevamente, la primary key se usa para obtener la fila de interés, por lo que se elimina una sola fila de la tabla. Al igual que la declaración _**UPDATE**_, se puede eliminar más de una fila, según las condiciones de su cláusula _**WHERE**_, y todas las filas serán eliminadas de omitirse la cláusula _**WHERE**_.

___

In [ ]:
################################################################################################################################